In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import probing_utils
import pickle

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [24]:
def get_model(checkpoint: int):
    model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
        checkpoint_index=checkpoint,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device)
    return model

NUM_CHECKPOINTS = 143
LAYER, NEURON = 3, 669
model = get_model(142)
german_data = haystack_utils.load_json_data("data/german_europarl.json")[:200]
english_data = haystack_utils.load_json_data("data/english_europarl.json")[:200]

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


In [ ]:
# Will take about 50GB of disk space for Pythia 70M models
def preload_models(NUM_CHECKPOINTS: int):
    for i in tqdm(range(NUM_CHECKPOINTS)):
        get_model(i)
preload_models(NUM_CHECKPOINTS)

In [43]:
def eval_loss(model, data):
    losses = []
    for prompt in data:
        loss = model(prompt, return_type="loss")
        losses.append(loss.item())
    return np.mean(losses)

def get_probe_performance(model, german_data, english_data, layer, neuron, plot=False):
    german_activations = haystack_utils.get_mlp_activations(german_data, layer, model, neurons=[neuron], mean=False)[:50000]
    english_activations = haystack_utils.get_mlp_activations(english_data, layer, model, neurons=[neuron], mean=False)[:50000]
    if plot:
        haystack_utils.two_histogram(german_activations.flatten(), english_activations.flatten(), "German", "English")
    return train_probe(german_activations, english_activations)

def train_probe(german_activations, english_activations):
    labels = np.concatenate([np.ones(len(german_activations)), np.zeros(len(english_activations))])
    activations = np.concatenate([german_activations.cpu().numpy(), english_activations.cpu().numpy()])
    scaler = preprocessing.StandardScaler().fit(activations)
    activations = scaler.transform(activations)
    x_train, x_test, y_train, y_test = train_test_split(activations, labels, test_size=0.2, random_state=SEED)
    probe = probing_utils.get_probe(x_train, y_train, max_iter=2000)
    f1, mcc = probing_utils.get_probe_score(probe, x_test, y_test)
    return f1, mcc

def eval_checkpoint(checkpoint: int):
    model = get_model(checkpoint)
    german_loss = eval_loss(model, german_data)
    f1, mcc = get_probe_performance(model, german_data, english_data, LAYER, NEURON)
    return [checkpoint, german_loss, f1, mcc]


In [14]:
model = get_model(NUM_CHECKPOINTS-1)
english_activations = {}
for layer in range(3, 4):
    english_activations[layer] = haystack_utils.get_mlp_activations(english_data, layer, model, mean=False)

MEAN_ACTIVATION_INACTIVE = english_activations[LAYER][:, NEURON].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURON] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER}.mlp.hook_post', deactivate_neurons_hook)]

print(MEAN_ACTIVATION_INACTIVE)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


  0%|          | 0/200 [00:00<?, ?it/s]

tensor(-0.0829, device='cuda:0')


In [ ]:
def get_layer_probe_performance(model, checkpoint, layer):
    english_activations = haystack_utils.get_mlp_activations(english_data[:31], layer, model, mean=False, disable_tqdm=True)[:10000]
    german_activations = haystack_utils.get_mlp_activations(german_data[:30], layer, model, mean=False, disable_tqdm=True)[:10000]
    neuron_labels = [f'C{checkpoint}L{layer}N{i}' for i in range(model.cfg.d_mlp)]
    mean_english_activations = english_activations.mean(0).cpu().numpy()
    mean_german_activations = german_activations.mean(0).cpu().numpy()
    f1s = []
    mccs= []
    for neuron in range(model.cfg.d_mlp):
        f1, mcc = train_probe(german_activations[:, neuron].unsqueeze(-1), english_activations[:, neuron].unsqueeze(-1))
        f1s.append(f1)
        mccs.append(mcc)
    df = pd.DataFrame({"Label": neuron_labels, "Neuron": [i for i in range(model.cfg.d_mlp)], "F1": f1s, "MCC": mccs, "MeanGermanActivation": mean_german_activations, "MeanEnglishActivation": mean_english_activations})
    df["Checkpoint"] = checkpoint
    df["Layer"] = layer
    return df

dfs = []
checkpoints = list(range(40)) + [40,50,60,70,80,90,100, 110, 120, 130, 140]
with tqdm(total=len(checkpoints)*model.cfg.n_layers) as pbar:
    for checkpoint in checkpoints:
        model = get_model(checkpoint)
        for layer in range(model.cfg.n_layers):
            tmp_df = get_layer_probe_performance(model, checkpoint, layer)
            dfs.append(tmp_df)
            with open("data/layer_probe_performance.pkl", "wb") as f:
                pickle.dump(dfs, f)
            pbar.update(1)

In [115]:
df = pd.concat(dfs)
df["NeuronLabel"] = df.apply(lambda row: f"L{row['Layer']}N{row['Neuron']}", axis=1)

,Label,Neuron,F1,MCC,MeanGermanActivation,MeanEnglishActivation,Checkpoint,Layer
0,C0L0N0,0,0.414037,0.009989,0.132858,0.117383,0,0
1,C0L0N1,1,0.437648,0.059326,0.158821,0.107143,0,0
2,C0L0N2,2,0.505419,-0.004196,0.103538,0.108237,0,0
3,C0L0N3,3,0.572180,0.021733,0.099450,0.143337,0,0
4,C0L0N4,4,0.582425,0.060994,0.152917,0.224428,0,0


In [117]:
checkpoints = []
top_probe = []
for checkpoint in df["Checkpoint"].unique():
    tmp_df = df[df["Checkpoint"] == checkpoint]
    top_probe.append(tmp_df["MCC"].max())
    checkpoints.append(checkpoint)
px.line(x=checkpoints, y=top_probe, title="Top Probe MCC by Checkpoint")

In [127]:
neurons = df[df["MCC"] > 0.7][["NeuronLabel", "MCC"]].copy()
neurons = neurons.sort_values(by="MCC", ascending=False)
print(len(neurons["NeuronLabel"].unique()))
good_neurons = neurons["NeuronLabel"].unique()[:50]

431


In [143]:
def get_mean_english(df, neuron, layer, checkpoint):
    label = f"C{checkpoint}L{layer}N{neuron}"
    df = df[df["Label"]==label]["MeanEnglishActivation"].item()
    return df

get_mean_english(df, 669, 3, 140)

-0.08759497106075287

In [138]:
df.head()

,Label,Neuron,F1,MCC,MeanGermanActivation,MeanEnglishActivation,Checkpoint,Layer,NeuronLabel
0,C0L0N0,0,0.414037,0.009989,0.132858,0.117383,0,0,L0N0
1,C0L0N1,1,0.437648,0.059326,0.158821,0.107143,0,0,L0N1
2,C0L0N2,2,0.505419,-0.004196,0.103538,0.108237,0,0,L0N2
3,C0L0N3,3,0.572180,0.021733,0.099450,0.143337,0,0,L0N3
4,C0L0N4,4,0.582425,0.060994,0.152917,0.224428,0,0,L0N4


In [144]:
ablation_data = []
checkpoints = list(range(0, NUM_CHECKPOINTS, 10))
print(checkpoints)
with tqdm(total=len(checkpoints)*len(good_neurons)) as pbar:
    for checkpoint in checkpoints:
        model = get_model(checkpoint)
        for neuron_name in good_neurons:
            layer, neuron = neuron_name[1:].split("N")
            layer, neuron = int(layer), int(neuron)
            english_activations = get_mean_english(df, neuron, layer, checkpoint)
            assert english_activations is not None
            def tmp_hook(value, hook):
                value[:, :, neuron] = english_activations
                return value
            tmp_hooks=[(f'blocks.{layer}.mlp.hook_post', tmp_hook)]
            original_loss = eval_loss(model, german_data)
            with model.hooks(tmp_hooks):
                ablated_loss = eval_loss(model, german_data)
            ablation_data.append([neuron_name, checkpoint, original_loss, ablated_loss])
            pbar.update(1)
    

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140]


  0%|          | 0/750 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.


...

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [ ]:
ablation_df = pd.DataFrame(ablation_data, columns=["Label", "Checkpoint", "OriginalLoss", "AblatedLoss"])
ablation_df.to_csv("data/checkpoint_ablation_data.csv")

In [98]:
max_mcc = df.groupby("NeuronLabel")["MCC"].max()
print(len(max_mcc[max_mcc < 0.1].index))
bad_neurons = []#max_mcc[max_mcc < 0.1].index[:10]
print(bad_neurons)

70
[]


In [99]:
px.line(df[df["NeuronLabel"].isin(good_neurons) | df["NeuronLabel"].isin(bad_neurons)], x="Checkpoint", y="MCC", color="NeuronLabel", title="Neurons with max MCC >= 0.85")

In [ ]:
# Filter neurons that are in the top 3 context neurons at any checkpoint
# Plot all these neurons as line plot with x being checkpoint and y being F1

data = []
for checkpoint in tqdm(range(NUM_CHECKPOINTS)):
    data.append(eval_checkpoint(checkpoint))

df = pd.DataFrame(data, columns=["checkpoint", "german_loss", "f1", "mcc"])

In [ ]:
ablation_losses = []
for checkpoint in tqdm(range(NUM_CHECKPOINTS)):
    model = get_model(checkpoint)
    with model.hooks(deactivate_neurons_fwd_hooks):
        ablated_loss = eval_loss(model, german_data)
    ablation_losses.append(ablated_loss)

In [ ]:
english_losses = []
for checkpoint in tqdm(range(NUM_CHECKPOINTS)):
    model = get_model(checkpoint)
    english_loss = eval_loss(model, english_data)
    english_losses.append(english_loss)

In [ ]:
df["english_loss"] = english_losses
df["ablation_loss"] = ablation_losses

In [103]:
df = pd.read_csv("data/checkpoint_eval.csv", index_col=0).reset_index()
df.head(5)

,checkpoint,german_loss,f1,mcc,english_loss,ablation_loss
0,0,11.026308,0.476689,-0.020651,11.051213,11.025739
1,1,11.026308,0.436970,-0.016553,11.051213,11.025739
2,2,11.026103,0.387683,0.037004,11.050580,11.025534
3,3,11.022297,0.408647,0.010090,11.038868,11.021735
4,4,10.972300,0.450796,-0.029007,10.886706,10.971827


In [101]:
end_prompt = " Vorschlägen"
all_ignore, _ = haystack_utils.get_weird_tokens(model, plot_norms=False)
common_tokens = haystack_utils.get_common_tokens(german_data, model, all_ignore, k=100)
prompts = haystack_utils.generate_random_prompts(end_prompt, model, common_tokens, 500, length=20)
print(model.to_str_tokens(prompts[0]))


  0%|          | 0/200 [00:00<?, ?it/s]

['h', 'e', ' V', ' Z', 'igen', 'chte', 'n', ' den', ' eine', ' dem', 'ig', ' ist', ' An', ' ein', ' Herr', 'ge', 'igen', ' Ab', ' Ber', ' nicht', ' V', 'orsch', 'lä', 'gen']


In [113]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot with 2 y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for german_loss and english_loss
fig.add_trace(go.Scatter(x=df['checkpoint'], y=df['german_loss'], name='German Loss'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['checkpoint'], y=df['english_loss'], name='English Loss'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['checkpoint'], y=df['ablation_loss'], name='Ablated German Loss'), secondary_y=False)

# Add traces for f1 and mcc
fig.add_trace(go.Scatter(x=df['checkpoint'], y=df['f1'], name='F1'), secondary_y=True)
fig.add_trace(go.Scatter(x=df['checkpoint'], y=df['mcc'], name='MCC'), secondary_y=True)

# Set y-axes titles
fig.update_yaxes(title_text="Loss", secondary_y=False)
fig.update_yaxes(title_text="Score", secondary_y=True)

fig.update_layout(title_text="German Loss, English Loss, L3N669 F1, and L3N669 MCC over Pythia 70M Checkpoints")
fig.update_layout(
    yaxis=dict(type='log'),
    yaxis2=dict(type='linear')
)


fig.show()


In [21]:
df.to_csv("data/checkpoint_eval.csv", index=False)

In [ ]:
# abltaion
# other context neurons in